In [1]:
'''
Requirements:
US Bureau of Labor Statistic API Key in config.py
Postgres credential in 

Sample Code Source:
https://www.bls.gov/developers/api_python.htm#python2

Data website:
https://data.bls.gov/timeseries/APU0000704111
'''
;

''

In [1]:
import requests
import json
import pandas as pd
import datetime as dt
from config import bls_api_key, username, passphrase
from sqlalchemy import create_engine

now_year = dt.date.today().year

def bacon_cpi_api(startyear =  now_year - 1, endyear = now_year):
    
    '''Default to since last year'''

    # Define variables
    series_id = 'APU0000704111'
    
    # Convert years to string
    startyear = str(startyear)
    endyear   = str(endyear)

    # Pulling API Data from US Bureau of Labor Statistics
    headers = {'Content-type': 'application/json'}
    data    = json.dumps({"seriesid": [series_id], "startyear"  : startyear, "endyear" : endyear, "registrationkey": bls_api_key})
    p       = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data = data, headers = headers)
    json_data = json.loads(p.text)
    
    # Print Status
    print(json_data["status"])
    
    # Print API return message if it's not empty
    if json_data['message']:
        print(json_data['message'])

    # Create a pandas DataFrame
    data_dict = {
        'year_month'  : [],
        'value' : [],
    }

    for item in json_data['Results']['series'][0]['data']:

        year_month  = item['year'] + item['period'].replace('M', '')
        value  = float(item['value'])
        
        data_dict['year_month'].append(year_month)
        data_dict['value'].append(value)

    data_pd = pd.DataFrame(data_dict)

    return data_pd

def store_cpi_data_to_db(data_pd):

    # Connect to Postgres
    rds_connection_string = f"{username}:{passphrase}@localhost:5432/bacon_db"
    engine = create_engine(f'postgresql://{rds_connection_string}')

    # Pull the last 24 data points from the database
    try:
        last_24m = pd.read_sql_query('SELECT * FROM bacon_cpi LIMIT 24', con=engine)
    except:
        # If the table bacon_cpi doesn't exsit in the database. Create one.
        data_pd.to_sql(name='bacon_cpi', con = engine, if_exists='append', index = False)
        
        print('A new table bacon_cpi has been created in the database bacon_db.')
        return
    
    # Store only the new data to the database
    for year_month in data_pd.year_month:
        if year_month not in last_24m.year_month.values:
        
            new_data = data_pd.loc[data_pd.year_month == year_month].copy()
        
            # Load Bacon CPI Data to Postgres
            new_data.to_sql(name='bacon_cpi', con = engine, if_exists='append', index = False)
        
            print(year_month, 'cpi data has been loaded to the database bacon_db.')
            
    return 

In [19]:
# Download and Transform Data
data_pd = bacon_cpi_api()

# Load and store data
store_cpi_data_to_db(data_pd)